# How to customize JSON output for Reports and Test Suites

In [ ]:
try:
    import evidently
except:
    !pip install git+https://github.com/evidentlyai/evidently.git

In [ ]:
import pandas as pd
import numpy as np

from sklearn import datasets

from evidently import ColumnMapping
from evidently.report import Report
from evidently.test_suite import TestSuite
from evidently.metric_preset import DataDriftPreset
from evidently.test_preset import DataDriftTestPreset
from evidently.core import AllDict

## Prepare Dataset

In [ ]:
data = datasets.fetch_california_housing(as_frame=True)
housing_data = data.frame

housing_data.rename(columns={'MedHouseVal': 'target'}, inplace=True)
housing_data['prediction'] = housing_data['target'].values + np.random.normal(0, 5, housing_data.shape[0])

In [ ]:
reference = housing_data.sample(n=5000, replace=False)
current = housing_data.sample(n=5000, replace=False)
current.iloc[:20, 3:5] = np.nan

## Report JSON customization: default json

In [ ]:
report = Report(metrics=[
    DataDriftPreset(), 
])

report.run(reference_data=reference, current_data=current)
report.json()

## Report dict/json customization: include_render parameter

In [ ]:
report.json(include_render=False)

## Report dict/json customization: include/exclude parameters

In [ ]:
# specify what to include in dict/json

report.as_dict(include={
    "DataDriftTable": {
        "drift_by_columns":{
            "target":{
                "column_name", "column_type", "drift_score"
            }}}})

In [ ]:
# specify what to include in dict/json, use True to include all the fields for a dictionary

report.as_dict(include={"DataDriftTable":{"drift_by_columns":{"target":True}}})

In [ ]:
# specify what to include in dict/json, use AllDict to apply filters to column-based results

report.as_dict(include={
    "DataDriftTable": {
        "drift_by_columns":AllDict({
                "column_name", "column_type", "drift_score"
            })}},
              exclude={"DataDriftTable": {"drift_by_columns":AllDict({
                 "column_type"
            })}})

In [ ]:
# specify what to include in dict/json, combine include and exclude

report.as_dict(
    include={"DatasetDriftMetric": {"share_of_drifted_columns"}},
    exclude={"DataDriftTable":{"drift_by_columns"}}
)

## TestSuite dict/json customization: include/exclude parameters

In [ ]:
# TestSuites support include in dict/json as well

test_suite = TestSuite(tests=[
    DataDriftTestPreset(),
])

test_suite.run(reference_data=reference, current_data=current)
test_suite.as_dict()

In [ ]:
test_suite.as_dict(exclude={"TestColumnDrift":{"parameters":True}})

In [ ]:
test_suite.as_dict(
    exclude={"TestColumnDrift":{"parameters":True}},
    include={"TestShareOfDriftedColumns":{"name", "status"}}
)

## TestSuite dict/json customization: include_metrics and include_render parameters

In [ ]:
# use include_metrics=True/False to include/exclude metrics data from Test Suite results 

test_suite.as_dict(include_metrics=True)

In [ ]:
# if you have included metrics in Test Suite output, you can customize what exactly to include
# use include_render=True/False to include/exclude render-related data from metrics 

test_suite.as_dict(include_metrics=True, include_render=False) 

In [ ]:
# if you have included metrics in Test Suite output, you can customize what exactly to include
# specify what metric fields to include in the output using include/exclude parameters, that you used before for Reports

test_suite.as_dict(include_metrics=True, include={
    "DataDriftTable": {
        "drift_by_columns":AllDict({
                "column_name", "column_type", "drift_score"
            })},
    "ColumnDriftMetric": {
        "column_name", "column_type", "drift_score"
    }
}) 